In [2]:
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [3]:
df_decisions = pd.read_csv('./stock_decisions.csv');
df_decisions.head()

,Current Value of Stock,Expected Value of Stock,Error in expected value of stock,Percentage of portfolio,Percentage of sector,Sector of stock,Buy/Sell/Keep,Positive,Negative,Neutral,Total,Neutral_Pos,Neutral_Neg
0,964.00,957.550,13.32,3.811099,10.530121,IT,Keep,37.088734,7.577268,1003,55.333998,37.088734,14.260219
1,958.00,959.830,13.74,4.363554,7.343906,IT,Keep,38.159879,11.085973,1326,50.754148,38.159879,22.348774
2,954.21,975.425,11.87,0.749470,10.261361,IT,Buy,36.168582,10.727969,1305,53.103448,36.168582,19.641470
3,970.00,982.610,20.11,2.028236,18.807057,IT,Sell,47.939017,7.227555,1771,44.833427,47.939017,24.411076
4,975.64,992.115,11.00,1.296318,14.393310,IT,Buy,34.628045,11.586570,1519,53.785385,34.628045,19.818134


In [4]:
df_decisions.drop('Sector of stock', axis=1, inplace=True)

In [5]:
one_hot_encoded = pd.get_dummies(df_decisions['Buy/Sell/Keep'], prefix='Buy/Sell/Keep')
one_hot_encoded = one_hot_encoded.astype(int)

In [6]:
df_decision_one_hot = df_decisions.drop(labels = 'Buy/Sell/Keep', axis=1)
df_decision_one_hot = pd.concat([df_decision_one_hot, one_hot_encoded], axis=1)

In [10]:
df_decision_one_hot.head()

,Current Value of Stock,Expected Value of Stock,Error in expected value of stock,Percentage of portfolio,Percentage of sector,Positive,Negative,Neutral,Total,Neutral_Pos,Neutral_Neg,Buy/Sell/Keep_Buy,Buy/Sell/Keep_Keep,Buy/Sell/Keep_Sell
0,964.00,957.550,13.32,3.811099,10.530121,37.088734,7.577268,1003,55.333998,37.088734,14.260219,0,1,0
1,958.00,959.830,13.74,4.363554,7.343906,38.159879,11.085973,1326,50.754148,38.159879,22.348774,0,1,0
2,954.21,975.425,11.87,0.749470,10.261361,36.168582,10.727969,1305,53.103448,36.168582,19.641470,1,0,0
3,970.00,982.610,20.11,2.028236,18.807057,47.939017,7.227555,1771,44.833427,47.939017,24.411076,0,0,1
4,975.64,992.115,11.00,1.296318,14.393310,34.628045,11.586570,1519,53.785385,34.628045,19.818134,1,0,0


In [14]:
kmeans = KMeans(n_clusters=10)
kmeans.fit(df_decision_one_hot)

KMeans(n_clusters=10)

In [15]:
centroids = kmeans.cluster_centers_